In [1]:
import json
import os
import shutil

def merge_train_splits(split_files, annotation_dir, output_dir):
    merged_annotations = []
    categories = None

    for file_name in split_files:
        path = os.path.join(annotation_dir, file_name)
        with open(path, 'r') as f:
            data = json.load(f)
            if categories is None:
                categories = data.get("categories", {})
            merged_annotations.extend(data.get("annotations", []))

    merged_data = {
        "categories": categories,
        "annotations": merged_annotations
    }

    output_train = os.path.join(output_dir, "20250512_annotations_train.json")
    with open(output_train, 'w') as f:
        json.dump(merged_data, f, indent=2)

    print(f"✅ Trainファイル作成: {output_train}（{len(merged_annotations)}件）")


def copy_val_split(annotation_dir, output_dir):
    src = os.path.join(annotation_dir, "20250512_annotations_split_0.json")
    dst = os.path.join(output_dir, "20250512_annotations_val.json")
    shutil.copyfile(src, dst)
    print(f"✅ Valファイルコピー: {dst}")


if __name__ == "__main__":
    annotation_dir = "/srv/shared/data/ego4d/annotations"
    output_dir = "/home/ollo/videomae-clean"

    # 必要なら出力先を作成
    os.makedirs(output_dir, exist_ok=True)

    # train を split_1〜3 からマージ
    split_files = [
        "20250512_annotations_split_1.json",
        "20250512_annotations_split_2.json",
        "20250512_annotations_split_3.json"
    ]
    merge_train_splits(split_files, annotation_dir, output_dir)

    # val を split_0 からコピー
    copy_val_split(annotation_dir, output_dir)


✅ Trainファイル作成: /home/ollo/videomae-clean/20250512_annotations_train.json（64955件）
✅ Valファイルコピー: /home/ollo/videomae-clean/20250512_annotations_val.json


In [1]:
import torch
print(torch.__version__)


2.3.0+cu118


In [2]:
!pip install psutil GPUtil


  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'GPUtil' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'GPUtil'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for GPUtil: filename=gputil-1.4.0-py3-none-any.whl size=7433 sha256=fc4fa0ca274f5513c03ca9cb8db3df8c9f99d660e6360b78f6132551945ca208
  Stored in directory: /home/ollo/.cache/pip/wheels/2b/4d/8f/55fb4f7b9b591891e8d3f72977c4ec6c7763b39c19f0861595
Successfully built GPUtil


In [3]:
import platform
import psutil
import torch
import GPUtil

# OS情報
print("=== OS 情報 ===")
print("システム:", platform.system())
print("リリース:", platform.release())
print("バージョン:", platform.version())

# CPU情報
print("\n=== CPU 情報 ===")
print("プロセッサ:", platform.processor())
print("物理コア数:", psutil.cpu_count(logical=False))
print("論理コア数:", psutil.cpu_count(logical=True))

# メモリ情報
print("\n=== メモリ情報 ===")
mem = psutil.virtual_memory()
print(f"合計メモリ: {mem.total / 1024**3:.2f} GB")

# PyTorchバージョンとCUDA
print("\n=== PyTorch / CUDA 情報 ===")
print("PyTorch バージョン:", torch.__version__)
print("CUDA 利用可能:", torch.cuda.is_available())
print("CUDA バージョン:", torch.version.cuda)

# GPU情報（ある場合）
print("\n=== GPU 情報 ===")
if torch.cuda.is_available():
    gpus = GPUtil.getGPUs()
    for gpu in gpus:
        print(f"GPU 名称: {gpu.name}")
        print(f"メモリ合計: {gpu.memoryTotal} MB")
        print(f"使用率: {gpu.load*100:.1f}%")
else:
    print("GPU は検出されませんでした。")


=== OS 情報 ===
システム: Linux
リリース: 5.10.0-34-cloud-amd64
バージョン: #1 SMP Debian 5.10.234-1 (2025-02-24)

=== CPU 情報 ===
プロセッサ: 
物理コア数: 8
論理コア数: 16

=== メモリ情報 ===
合計メモリ: 62.81 GB

=== PyTorch / CUDA 情報 ===
PyTorch バージョン: 2.3.0+cu118
CUDA 利用可能: True
CUDA バージョン: 11.8

=== GPU 情報 ===
GPU 名称: NVIDIA L4
メモリ合計: 23034.0 MB
使用率: 0.0%
